## DATASET BANKING

En función de las diferentes características de un solicitante de un préstamo, decicidir si se le concede dicho préstamo (1) o no (0)

In [18]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss

In [19]:
# Cargamos el CSV en un dataframe
df = pd.read_csv("banking.csv")

# Imprimimos información del csv para ver con que datos trabajamos
# print(df.info())
# print(df.describe())

# Imprimimos los valores únicos de las columnas para saber que preprocesamiento aplicar sobre cada uno (OnehotEncoder, StandarScaler...)
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

# Resumir la columna educación
df["education"] = np.where((df["education"] == 'basic.4y') | (df["education"] == 'basic.6y') | (df["education"] == 'basic.9y'), 
                           'Basic', 
                           df["education"])

age: [44 53 28 39 55 30 37 36 27 34 41 33 26 52 35 40 32 49 38 47 46 29 54 42
 72 48 43 56 31 24 68 59 50 45 25 57 63 58 60 64 51 23 20 74 80 61 62 75
 21 82 77 70 76 73 66 22 71 19 79 88 65 67 81 18 84 69 98 85 83 78 92 86
 94 17 91 89 87 95]
job: ['blue-collar' 'technician' 'management' 'services' 'retired' 'admin.'
 'housemaid' 'unemployed' 'entrepreneur' 'self-employed' 'unknown'
 'student']
marital: ['married' 'single' 'divorced' 'unknown']
education: ['basic.4y' 'unknown' 'university.degree' 'high.school' 'basic.9y'
 'professional.course' 'basic.6y' 'illiterate']
default: ['unknown' 'no' 'yes']
housing: ['yes' 'no' 'unknown']
loan: ['no' 'yes' 'unknown']
contact: ['cellular' 'telephone']
month: ['aug' 'nov' 'jun' 'apr' 'jul' 'may' 'oct' 'mar' 'sep' 'dec']
day_of_week: ['thu' 'fri' 'tue' 'mon' 'wed']
duration: [ 210  138  339 ... 2260 1662 1490]
campaign: [ 1  3  2  8  5  4 25 11 12 18  6 17  7 20 16 14 10  9 19 29 13 40 15 34
 22 24 41 21 23 39 28 27 31 35 26 30 32 43 33 42 56 37

Mediante la información obtenida al analizar el contenido y tipo de las columnas llegamos a la conclusión de que las variables categóricas no poseen nignuna un orden de prioridad o peso, por lo que emplearemos el OneHot Encoder para la encodificación de todas ellas.

In [ ]:
# Una vez encontrados las variables categóricas y decidido el método de encodeamineto lo aplicamos
df = pd.get_dummies(df, columns=["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"], drop_first=True)  

# Selección de las variables objetivo e independientes
X = df.drop(["y"], axis=1)
y = df["y"]

# Preprocesamos los datos mediante StandarScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividimos los datos en entrenamineto y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --------------------
# MÉTODOS DE BALANCEO DE CLASES
# --------------------
# 1. Sobremuestreo aleatorio
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 3. Submuestreo aleatorio
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# 4. NearMiss (submuestreo basado en distancia)
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

# --------------------
# EVALUACIÓN DE MÉTODOS DE BALANCEO
# --------------------
resultados_balanceo = {}

def evaluar_balanceo(X_train_resampled, y_train_resampled, metodo):
    # Creación del modelo de RL
    model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)
    # Entreno el modelo con los datos de cada modelo
    model.fit(X_train_resampled, y_train_resampled)
    # Realizo predicciones con el modelo
    y_pred = model.predict(X_test)
    # Calculo la precisión del modelo sacando las métricas
    f1 = f1_score(y_test, y_pred) * 100
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) * 100
    resultados_balanceo[metodo] = (f1, auc)
    # Imprimimos los resultados
    print(f"\nResultados con {metodo}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred) * 100:.2f}%")
    print(f"F1-Score: {f1:.2f}%")
    print(f"AUC-ROC: {auc:.2f}%")

evaluar_balanceo(X_train_ros, y_train_ros, "Sobremuestreo Aleatorio")
evaluar_balanceo(X_train_smote, y_train_smote, "SMOTE")
evaluar_balanceo(X_train_rus, y_train_rus, "Submuestreo Aleatorio")
evaluar_balanceo(X_train_nm, y_train_nm, "NearMiss")

# Seleccionar el mejor método de balanceo
mejor_metodo = max(resultados_balanceo, key=lambda k: resultados_balanceo[k])
print(f"\nEl mejor método de balanceo es: {mejor_metodo} con F1-Score: {resultados_balanceo[mejor_metodo][0]:.2f}% y AUC-ROC: {resultados_balanceo[mejor_metodo][1]:.2f}%")

# Obtener los datos balanceados del mejor método encontrado
if mejor_metodo == "Sobremuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_ros, y_train_ros
elif mejor_metodo == "SMOTE":
    X_train_resampled, y_train_resampled = X_train_smote, y_train_smote
elif mejor_metodo == "Submuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_rus, y_train_rus
elif mejor_metodo == "NearMiss":
    X_train_resampled, y_train_resampled = X_train_nm, y_train_nm

# --------------------
# MÉTODO 1: Ajuste manual de hiperparámetros
# --------------------
manual_model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)  # Ejemplo de hiperparámetros ajustados manualmente
manual_model.fit(X_train_resampled, y_train_resampled)  # Entrenar modelo

# Predicciones
y_pred_manual = manual_model.predict(X_test)

# Evaluación del modelo ajustado manualmente
print("\nResultados con ajuste manual de hiperparámetros:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, manual_model.predict_proba(X_test)[:,1]) * 100:.2f}%")

# --------------------
# MÉTODO 2: Ajuste automático con GridSearchCV
# --------------------
parametros = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],  # Diferentes valores de regularización
    "solver": ["liblinear", "lbfgs"]  # Diferentes algoritmos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=500), parametros, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)  # Entrenar búsqueda de hiperparámetros

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Evaluación del mejor modelo
print("\nResultados con ajuste automático de hiperparámetros:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor Accuracy en validación cruzada: {grid_search.best_score_ * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_best) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_best) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) * 100:.2f}%")


# --------------------
# ENTRENAMIENTO FINAL Y EVALUACIÓN
# --------------------

# Entrenar el mejor modelo con los datos balanceados
best_model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones finales con el mejor modelo y el mejor balanceo
y_pred_final = best_model.predict(X_test)

# Crear DataFrame de resultados
resultados = pd.DataFrame({
    "Real": y_test.values,
    "Predicción": y_pred_final
})

# Mapear valores 0 y 1 a etiquetas comprensibles
resultados["Real"] = resultados["Real"].map({0: "No clase y", 1: "clase y"})
resultados["Predicción"] = resultados["Predicción"].map({0: "No clase y", 1: "clase y"})

# Mostrar las primeras filas de la tabla
print(resultados.head(10))


Resultados con Sobremuestreo Aleatorio:
Accuracy: 85.64%
Precision: 43.76%
Recall: 89.29%
F1-Score: 58.74%
AUC-ROC: 93.52%

Resultados con SMOTE:
Accuracy: 86.17%
Precision: 44.76%
Recall: 88.76%
F1-Score: 59.51%
AUC-ROC: 93.41%

Resultados con Submuestreo Aleatorio:
Accuracy: 85.43%
Precision: 43.40%
Recall: 89.61%
F1-Score: 58.48%
AUC-ROC: 93.39%

Resultados con NearMiss:
Accuracy: 74.36%
Precision: 29.32%
Recall: 87.91%
F1-Score: 43.98%
AUC-ROC: 88.33%

El mejor método de balanceo es: SMOTE con F1-Score: 59.51% y AUC-ROC: 93.41%

Resultados con ajuste manual de hiperparámetros:
Accuracy: 86.17%
Precision: 44.76%
Recall: 88.76%
F1-Score: 59.51%
AUC-ROC: 93.41%

Resultados con ajuste automático de hiperparámetros:
Mejores parámetros: {'C': 10, 'solver': 'lbfgs'}
Mejor Accuracy en validación cruzada: 88.52%
Accuracy: 86.17%
Precision: 44.74%
Recall: 88.44%
F1-Score: 59.42%
AUC-ROC: 93.39%
   Real  Predicción
0     0           0
1     1           0
2     1           1
3     0          